In [1]:
__nbid__ = '0063'
__author__ = 'Antares Team <antares@noirlab.edu>'
__version__ = '20240601' # yyyymmdd
__datasets__ = ['']
__keywords__ = ['ANTARES', 'Gravitational wave events']

# Accessing gravitational wave events using ANTARES

*ANTARES Team.*

### Table of contents
* [Goals & notebook summary](#goals)
* [Disclaimer & attribution](#attribution)
* [Imports & setup](#import)
* [Content](#chapter1)
    * [0. Background information on ANTARES](#content-background)
    * [1. What is a gravitational wave?](#content-grav-waves)
    * [2. How to access gravitational wave notices using the ANTARES devkit?](#content-grav-waves-devkit)
    * [3. How to develop a filter that uses gravitational wave data?](#content-grav-waves-filter)
    * [4. How to access gravitational wave notices using the ANTARES client?](#content-grav-waves-client)

<a class="anchor" id="goals"></a>
# Goals
To show how to access gravitational wave data from [GCN](https://gcn.nasa.gov/) in ANTARES.

# Summary
This Jupyter notebook is an example of how to access gravitational wave notices [GCN](https://gcn.nasa.gov/) using the ANTARES devkit and client

<a class="anchor" id="attribution"></a>
# Disclaimer & attribution

Disclaimers
-----------
Note that using the Astro Data Lab constitutes your agreement with our minimal [Disclaimers](https://datalab.noirlab.edu/disclaimers.php).

Acknowledgments
---------------
If you use **Astro Data Lab** in your published research, please include the text in your paper's Acknowledgments section:

_This research uses services or data provided by the Astro Data Lab, which is part of the Community Science and Data Center (CSDC) Program of NSF NOIRLab. NOIRLab is operated by the Association of Universities for Research in Astronomy (AURA), Inc. under a cooperative agreement with the U.S. National Science Foundation._

If you use **SPARCL jointly with the Astro Data Lab platform** (via JupyterLab, command-line, or web interface) in your published research, please include this text below in your paper's Acknowledgments section:

_This research uses services or data provided by the SPectra Analysis and Retrievable Catalog Lab (SPARCL) and the Astro Data Lab, which are both part of the Community Science and Data Center (CSDC) Program of NSF NOIRLab. NOIRLab is operated by the Association of Universities for Research in Astronomy (AURA), Inc. under a cooperative agreement with the U.S. National Science Foundation._

In either case **please cite the following papers**:

* Data Lab concept paper: Fitzpatrick et al., "The NOAO Data Laboratory: a conceptual overview", SPIE, 9149, 2014, https://doi.org/10.1117/12.2057445

* Astro Data Lab overview: Nikutta et al., "Data Lab - A Community Science Platform", Astronomy and Computing, 33, 2020, https://doi.org/10.1016/j.ascom.2020.100411

If you are referring to the Data Lab JupyterLab / Jupyter Notebooks, cite:

* Juneau et al., "Jupyter-Enabled Astrophysical Analysis Using Data-Proximate Computing Platforms", CiSE, 23, 15, 2021, https://doi.org/10.1109/MCSE.2021.3057097

If publishing in a AAS journal, also add the keyword: `\facility{Astro Data Lab}`

And if you are using SPARCL, please also add `\software{SPARCL}` and cite:

* Juneau et al., "SPARCL: SPectra Analysis and Retrievable Catalog Lab", Conference Proceedings for ADASS XXXIII, 2024
https://doi.org/10.48550/arXiv.2401.05576

The NOIRLab Library maintains [lists of proper acknowledgments](https://noirlab.edu/science/about/scientific-acknowledgments) to use when publishing papers using the Lab's facilities, data, or services.

<a class="anchor" id="import"></a>
# Imports and setup

In [2]:
import antares.devkit as dk
dk.init()

Loading ANTARES from /data0/sw/antares-kernel/lib/python3.9/site-packages/antares/__init__.py

        _    _   _ _____  _    ____  _____ ____
       / \  | \ | |_   _|/ \  |  _ \| ____/ ___|
      / _ \ |  \| | | | / _ \ | |_| |  _| \___ \\
     / ___ \| |\  | | |/ ___ \|  _ /| |___ ___| |
    /_/   \_\_| \_| |_/_/   \_\_| \_\_____|____/   v2.11.0
    
2024-06-01 10:26:12,388 - WARNING MainThread settings.py:setup_prometheus:124 - Prometheus failed to start with [Errno 98] Address already in use
2024-06-01 10:26:12,484 - WARNING MainThread settings.py:setup_prometheus:124 - Prometheus failed to start with [Errno 98] Address already in use


Jaeger tracer already initialized, skipping


Testing loading a random Locus with `dk.get_locus()`...

ANTARES v2.11.0 DevKit is ready!
Website: https://antares.noirlab.edu
Documentation: https://nsf-noirlab.gitlab.io/csdc/antares/antares/



/data0/sw/antares-kernel/lib/python3.9/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)


<a class="anchor" id="content-background"></a>
# 0. Background information on ANTARES

ANTARES receives alerts from surveys in real-time and sends them through a processing pipeline. The pipeline contains the following stages:

1. Associate the alert with the nearest point of known past measurements within a 1" radius. We call this a Locus.
2. Discard alerts with a high probability of being false detections.
3. Discard alerts with poor image quality.
4. Associate gravitational wave events from [GCN](https://gcn.nasa.gov/) to Locus.
5. Look up associated objects in our catalogs.
6. Update watch lists.
7. Execute filters on Locus.
8. Send Locus to user Kafka topics.


Filters are Python functions that take a Locus object as a single parameter. Functions on the Locus provide access to the alerts properties, the data from past alerts on the Locus, gravitational wave events on the Locus, and the associated catalog objects. This information can be used to characterize or classify the Locus. The Locus also provides functions to set new properties on the Alert and Locus objects, and to send the Locus to a specific Kafka stream.

In [3]:
# First we retrieve a locus
locus = dk.get_locus("ANT2020ho42c", include_gravitational_wave_data=True)

<a class="anchor" id="content-grav-waves"></a>
# 1. What is a gravitational wave?

Gravitational waves are ripples in space.  The current suite of gravitational wave detectors are sensitive to the coalescence of binary systems with massive compact objects, such as neutron stars and
black holes.

<a class="anchor" id="content-grav-waves-devkit"></a>
# 2. How to access gravitational wave notices using the ANTARES devkit?

We can access gravitational wave events associated with a locus through the `grav_wave_events_metadata` dictionary

In [4]:
for gracedb_id, grav_wave_notice in locus.grav_wave_events_metadata.items():
    print(gracedb_id, grav_wave_notice.event_datetime)

S231025ap 2023-10-25 14:23:06.141000
S231025aw 2023-10-25 15:07:18.932000
S231025az 2023-10-25 16:03:30.074000
S231025c 2023-10-25 03:29:30.125000
S231025p 2023-10-25 05:38:55.316000
S231025t 2023-10-25 06:16:12.235000
S231026n 2023-10-26 07:11:39.708000
S231026z 2023-10-26 12:54:28.507000
S231027bk 2023-10-27 20:01:09.100000
S231027m 2023-10-27 02:15:46.238000
S231027r 2023-10-27 03:51:15.755000
S231028aw 2023-10-28 14:00:36.384000
S231028br 2023-10-28 17:34:20.951000
S231028r 2023-10-28 06:59:50.019000
S231029ah 2023-10-29 14:19:37.578000
S231029ai 2023-10-29 14:51:39.214000
S231029at 2023-10-29 19:48:41.864000
S231029bd 2023-10-29 22:13:58.142000
S231029e 2023-10-29 00:54:46.395000
S231029n 2023-10-29 07:44:33.518000
S231029y 2023-10-29 11:15:08.754000
S231030ad 2023-10-30 06:25:23.301000
S231030bd 2023-10-30 15:05:30.555000
S231030be 2023-10-30 15:05:30.437000
S231030r 2023-10-30 02:12:04.512000
S231030t 2023-10-30 02:26:50.708000
S231030v 2023-10-30 02:51:32.206000
S231103aa 2023-

For more information and a complete description of the gravitational
wave notice class, see https://nsf-noirlab.gitlab.io/csdc/antares/antares/grav_waves.html

We can access gravitational wave events associated with an alert through the `grav_wave_events` variable

In [5]:
for event in locus.alerts[-1].grav_wave_events:
    print(event)

{'gracedb_id': 'S231103aa', 'contour_level': 99.43336421502327, 'contour_area': 17588.237629045216}
{'gracedb_id': 'S231104j', 'contour_level': 85.33401140757233, 'contour_area': 1631.7966697018112}
{'gracedb_id': 'S231104m', 'contour_level': 94.29495236819994, 'contour_area': 14449.279395955451}
{'gracedb_id': 'S231104s', 'contour_level': 93.68839355615805, 'contour_area': 15322.144786975066}
{'gracedb_id': 'S231106k', 'contour_level': 99.08665701692547, 'contour_area': 16916.80271287628}
{'gracedb_id': 'S231106r', 'contour_level': 89.01270487170156, 'contour_area': 4625.347278758741}
{'gracedb_id': 'S231107a', 'contour_level': 94.34758023019685, 'contour_area': 11236.463322087104}
{'gracedb_id': 'S231108l', 'contour_level': 95.66782505934242, 'contour_area': 3123.8509474759635}
{'gracedb_id': 'S231108p', 'contour_level': 98.54951439973834, 'contour_area': 12085.828491040804}
{'gracedb_id': 'S231108x', 'contour_level': 87.52196496667342, 'contour_area': 2202.306525034102}
{'gracedb_id

We can use the `gracedb_id` property to search for an event in the treasuremap

In [6]:
def get_treasuremap_link(gracedb_id: str):
    return f"https://treasuremap.space/alerts?graceids={gracedb_id}"

print(get_treasuremap_link("S231103aa"))

https://treasuremap.space/alerts?graceids=S231103aa


<a class="anchor" id="content-grav-waves-filter"></a>
# 3. How to develop a filter that uses gravitational wave data?

When developing a filter using the devkit we can restrict the alerts that the filter will be ran on based on the probability region

In [7]:
class SomeFilter(dk.Filter):
    REQUIRED_GRAV_WAVE_PROB_REGION = 48.97257324004685

    def run(self, locus):
        print(f"Number of gravitational wave notices: {len(locus.grav_wave_events_metadata)}")

In [8]:
report = dk.run_filter(SomeFilter, locus)
print(report)

Number of gravitational wave notices: 50
{'locus_id': 'ANT2020ho42c', 'locus_data': FilterContext(locus_id="ANT2020ho42c"), 't': 4.9240000000949635e-05, 'new_locus_properties': {}, 'new_alert_properties': {}, 'new_tags': set(), 'raised_halt': False}


<a class="anchor" id="content-grav-waves-client"></a>
# 4. How to access gravitational wave notices using the ANTARES client?

For cases when we want to access a gravitational wave notice by ID we can use methods of the antares_client, described at https://nsf-noirlab.gitlab.io/csdc/antares/client/tutorial/searching.html

In [9]:
import antares_client

A gravitational wave notice can have multiple alerts, each of which can have a different notice type (for more information see https://emfollow.docs.ligo.org/userguide/content.html#notice-types). If we want to retrieve the last alert received, we can use the `get_latest_grav_wave_notices` method

In [10]:
gracedb_id = "S231030v"
gravitational_wave_notice_1 = antares_client.search.get_latest_grav_wave_notices(gracedb_id)

For cases where we want to access a particular gravitational wave notice at a specific time, we can specify the datetime of the notice and use the `get_grav_wave_notices` method

In [11]:
import datetime

In [12]:
notice_datetime = datetime.datetime(2023, 10, 30, 2, 56, 33)
gravitational_wave_notice_2 = antares_client.search.get_grav_wave_notices(gracedb_id, notice_datetime)

In [13]:
gravitational_wave_notice_1 == gravitational_wave_notice_2

True